In [3]:
import datetime as dt
import pytz
# from matplotlib import pyplot as plt
from django.db.models.query_utils import Q
import pandas as pd
from tqdm.notebook import tqdm
import openpyxl
import calendar

In [6]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [7]:
#hs_tabela = pd.read_excel("hs.xlsx", engine ='openpyxl')

In [10]:
road_companies=['Arteris Fernão Dias',
'Arteris Fernão Dias - Conserva',
'Arteris Fluminense',
'Arteris Fluminense - Conserva',
'Arteris Intervias',
'Arteris Litoral Sul',
'Arteris Litoral Sul - Conserva',
'Arteris Planalto Sul',
'Arteris Planalto Sul - Conserva',
'Arteris Regis Bittencourt',
'Arteris Régis Bittencourt - Conserva',
'Arteris Via Paulista',
'Arteris Via Paulista - Faixa de Domínio',
'Caminhos da Serra Gaúcha',
'CART',
'Dalba - Foz do Iguaçu',
'Dalba Guarapuava',
'Eixo SP',
'Eixo sp Faixa de Domínio / Topografia / qualidade',
'ELO Ecopistas',
'Elo Ecovias dos Imigrantes',
'ELO Rodovias do Tietê',
'Elo Rodovias dos Tamoios',
'ELO Rota das Bandeiras ',
'Elo Via Lagos',
'Entrevias',
'Garcia Monteiro',
'Modera ',
'Neovia Alcinópolis',
'Neovia Araquari 886/2020',
'Neovia Araranguá',
'Neovia Bocaína do Sul 329/2018',
'Neovia Contorno Sul',
'Neovia Cto Biguaçu',
'Neovia Diamantino',
'Neovia Lages 652/2019',
'Neovia Lagoa Vermelha',
'Neovia Planalto sul',
'Neovia Ponte Serrada',
'Neovia Rancho Queimado 080/2021',
'Neovia Régis Bittencourt',
'Neovia Tijucas',
'Pavicolet (Amapá)',
'Pavicolet (Amazonas)',
'Rota do Oeste',
'Sanson',
'Sinaliza Via (Municípios) ',
'VF Gomes - BR010',
'VF Gomes - Confresa',
'VF Gomes - Nova Lacerda',
'VF Gomes - Porto Espiridião',
'VF Gomes - Vila Bela',
'VF Gomes BR 158',
'Via Brasil']

In [4]:
road_companies = [x for x in Company.objects.filter(active=True).values_list('name',flat=True).order_by('name')]

In [5]:
road_companies=['Engie Umburanas']

In [6]:
today = dt.date.today().strftime('%m')
year = int(dt.date.today().strftime('%Y'))
meses = [int(today)-i for i,a in enumerate(range(2),start=1)]
meses.sort()
months_number=meses
meses,year

([10, 11], 2023)

In [7]:
last_days=[dt.date(year,a,calendar.monthrange(year, a)[1]).day for a in meses]
last_days

[31, 30]

In [8]:
def quantitativos_v2(months_name,company):
    sem_qtt =[]

    for n in months_name:
        com_qtt = 0
        apontamentos_totais = 0
        for occ in OccurrenceType.objects.filter(company__name = company).exclude(occurrence_kind='2'):
            autofill_keys = []
            try:
                autofill_keys = [
                    field["apiName"] for field in occ.form_fields["fields"] if "autofill" in field.keys()
                ]
            except:
                autofill_keys = [
                    field["api_name"] for field in occ.form_fields["fields"] if "autofill" in field.keys()
                ]
            size_occ=len(autofill_keys)            
            reportings=Reporting.objects.filter(
                occurrence_type=occ, 
                updated_at__month = n, 
                company__name = company, 
                updated_at__year=year
            ).exclude(
                occurrence_type__occurrence_kind='2'
            ).exclude(
                created_by__username='rlcs'
            ).exclude(
                created_by__username='cartcs'
            ).only(
                'form_data', 'occurrence_type'
            ).select_related(
                'occurrence_type'
            )
            
            reportings_keys=[
                len(a.keys()) for a in list(reportings.values_list('form_data', flat=True))
            ]
            com_qtt += len([a for a in reportings_keys if a > size_occ])
            apontamentos_totais += reportings.count()
        sem_qtt.append((apontamentos_totais - com_qtt)/apontamentos_totais if apontamentos_totais != 0 else 0)
    return sem_qtt

In [9]:
def dias_de_uso_v2(months_name, company):
    
    qtd_dias = []
#     dias = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    dias=last_days

    for index, mes in enumerate(months_name):

        a= []
        for n in range(1, dias[index]):
            a.append(
                Reporting.objects.filter(
                    company__name = company, 
                    updated_at__year=year, 
                    updated_at__month = mes,
                    updated_at__day=n
                ).exclude(
                    occurrence_type__occurrence_kind='2'
                ).exclude(
                    created_by__username='rlcs'
                ).exclude(
                    created_by__username='neoviacs'
                ).exclude(
                    created_by__username='arteriscs'
                ).only('uuid').count()
            )
        qtd_dias.append(a)

    
    quantidade = []
    porcen_dias= []
    
    for qtd in qtd_dias:
        quantidade.append(len([x for x in qtd if x != 0]))
        
    
    for index, qtd in enumerate(quantidade):
        b = qtd/dias[index]
        porcen_dias.append(b)
    return porcen_dias

In [10]:
# Não está em uso o que está vigente é o v2
def dias_de_uso_v3(months_name, company):
    
    porcen_dias = []
    dias=last_days

    for mes in months_name:

        a = []
        dias_do_mes = dias[mes-1]
        for n in range(1, dias_do_mes):
            a.append(
                Reporting.objects.filter(
                    company__name = company, 
                    updated_at__year=year, 
                    updated_at__month = mes,
                    updated_at__day=n
                ).exclude(
                    occurrence_type__occurrence_kind='2'
                ).exclude(
                    created_by__username='rlcs'
                ).exclude(
                    created_by__username='neoviacs'
                ).exclude(
                    created_by__username='arteriscs'
                ).only('uuid').count()
            )

        dias_com_apontamentos = len([x for x in a if x != 0])
        porcen_dias.append(dias_com_apontamentos/dias_do_mes)

    return porcen_dias

In [11]:
def equipes_ativas_v2(months_name, company):
    team_name = list(Firm.objects.filter(company__name = company).values_list('name', flat=True))
    
    equipes=[]
    for team in team_name:
        for i in months_name:
            equipes.append(
                Reporting.objects.filter(
                    company__name = company, 
                    updated_at__year=year, 
                    updated_at__month= i,
                    firm__name = team
                ).exclude(
                    occurrence_type__occurrence_kind='2'
                ).exclude(
                    created_by__username='rlcs'
                ).exclude(
                    created_by__username='neoviacs'
                ).exclude(
                    created_by__username='arteriscs'
                ).only('uuid').count()
            )
    

    soma = []
    for team in team_name:

        soma.append(Reporting.objects.filter(company__name = company, updated_at__year=year,firm__name = team).exclude(occurrence_type__occurrence_kind='2').exclude(created_by__username='rlcs').exclude(created_by__username='neoviacs').exclude(created_by__username='arteriscs').only('uuid').count())

    porcen = []
    for month in months_name:
        firm_moves=[]
        objects=['Reporting', 'Job']
        for firm in team_name:
            num_moves = 0
            for item in objects:
                num_moves+=eval(item).history.filter(history_date__year=year, history_date__month=month, company__name= company, firm__name=firm).only('uuid').count()
            firm_moves.append((num_moves, firm))

        firm_moves.sort(key=lambda tup: tup[0], reverse=True)
        active_firms=len([firm for firm in firm_moves if firm[0]>0])
        porcen.append(active_firms/len(firm_moves) if len(firm_moves) != 0 else 0)
    
    return porcen

In [12]:
def variacao(months):
    taxa_variacao = []
    for i,m in enumerate(months):
        y = months[i]
        b = months
        c = b.index(y) 
        if c == 0:
            taxa_variacao.append(0)
        if y > b[c-1] and c != 0:
            m = (y-b[c-1])/y
            taxa_variacao.append(m)
        if y == b[c-1] and c != 0:
            i = y*0
            taxa_variacao.append(i)
        if y < b[c-1] and c!= 0 :
            n = (y -b[c-1])/b[c-1]
            taxa_variacao.append(n)
    return taxa_variacao


In [13]:
def qtt_nota(nota):
    a = 0
    if nota <0.05:
        a = 1
    if nota >= 0.05 and nota <0.15:
        a = 0.65
    if nota >= 0.15 and nota < 0.3:
        a = 0.3
    if nota >= 0.3:
        a = 0
    return a

In [14]:
def uso_nota(nota):
    a = 0
    if nota <0.4:
        a = 0
    if nota >= 0.4 and nota <0.65:
        a = 0.3
    if nota >= 0.65 and nota < 0.8:
        a = 0.65
    if nota > 0.8:
        a = 1
    return a

In [15]:
def equipe_nota(nota):
    a = 0
    if nota > 0.6:
        a = 1
    if nota >0.45 and nota <=0.6:
        a = 0.65
    if nota > 0.3 and nota <= 0.45:
        a = 0.3
    if nota <= 0.3:
        a = 0
    return a

In [16]:
def variacao_nota(nota):
    a= 0
    if nota >= 0.25:
        a = 1
    if nota >= 0.15 and nota <0.25:
        a = 0.75
    if nota >= -0.15 and nota < 0.15:
        a = 0.5
    if nota >= -0.25 and nota < -0.15:
        a = 0.5
    if nota < -0.25:
        a = 0
    return a

In [17]:
uso_sistema = []
sem_quantitativos = []
equipes = []
variacao_dias = []
for company in tqdm(road_companies):
    months=[]
    print(company)
    for n in months_number:
        months.append(
#             Reporting.objects.filter(
#                 company__name = company, 
#                 created_at__year=year, 
#                 created_at__month=n
#             ).exclude(
#                 occurrence_type__occurrence_kind='2'
#             ).exclude(
#                 created_by__username='rlcs'
#             ).exclude(
#                 created_by__username='neoviacs'
#             ).exclude(
#                 created_by__username='arteriscs'
#             ).only('uuid').count()
            Reporting.objects.filter(
                company__name = company, 
                updated_at__year=year, 
                updated_at__month=n
            ).exclude(
                occurrence_type__occurrence_kind='2'
            ).exclude(
                created_by__username='rlcs'
            ).exclude(
                created_by__username='neoviacs'
            ).exclude(
                created_by__username='arteriscs'
            ).only('uuid').count()
        )
    uso_sistema.append(dias_de_uso_v2(months_number, company))
    sem_quantitativos.append(quantitativos_v2(months_number, company))
    equipes.append(equipes_ativas_v2(months_number, company))
    variacao_dias.append(variacao(months)) 
    #print(months)
#     print(f' dias de uso: {dias_de_uso_v2(months_number, company)} \n sem quantitativos: {quantitativos_v2(months_number, company)} \n equipes ativas: {equipes_ativas_v2(months_number, company)} \n variação dias: {variacao(months)} ')

  0%|          | 0/1 [00:00<?, ?it/s]

Engie Umburanas


In [18]:
hs_month = []
hs_month_export = {}
for i in tqdm(range(len(months_number))):
    hs = []
    for n in range(len(road_companies)):
        print(road_companies[n])
        if uso_sistema[n][i] == 0:
            hs.append(0)
        else:
            hs.append(
                35*(uso_nota(uso_sistema[n][i])) 
                + 15*(qtt_nota(sem_quantitativos[n][i])) 
                + 15*(equipe_nota(equipes[n][i]))
                + 35*(variacao_nota(variacao_dias[n][i]))
            )
            print(35 * uso_nota(uso_sistema[n][i]))
            print(15 * qtt_nota(sem_quantitativos[n][i]))
            print(15 * equipe_nota(equipes[n][i]))
            print(35 * variacao_nota(variacao_dias[n][i]))
        if not hs_month_export.get(road_companies[n]):
            hs_month_export[road_companies[n]] = []
        hs_month_export[road_companies[n]].append(hs[-1])

    hs_month.append(hs)

  0%|          | 0/2 [00:00<?, ?it/s]

Engie Umburanas
0
15
4.5
17.5
Engie Umburanas
0
15
4.5
0


In [19]:
hs_month_export

{'Engie Umburanas': [37.0, 19.5]}

In [20]:
hs_novo = []
for i in hs_month_export.values():
    hs_novo.append(i[-1])

In [21]:
hs_novo

[19.5]

In [22]:
#with pd.ExcelWriter("hs2022piloto" + '.xlsx') as writer:
#    hs_excel = hs_tabela.to_excel(writer, index = False, sheet_name = "Página 1")

In [23]:
df= pd.DataFrame(data=hs_month_export)
df = df.transpose()

In [24]:
df.columns=['Penúltimo Mês','Último Mês']
df

,Penúltimo Mês,Último Mês
Engie Umburanas,37.0,19.5


In [55]:
df.to_excel('hsOut2023.xlsx')